In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import matplotlib.style as mplstyle
%matplotlib inline
mplstyle.use('fivethirtyeight')

 # Categorical Data

Ok first the simple stuff. How do you get the unique values and their frequencies?

In [2]:
values = pd.Series('apple orange apple apple'.split() * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

Check out this cool thing.

In [5]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(['apple', 'orange'])

In [6]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim

0     apple
1    orange
dtype: object

In [8]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

That's called a *categorical* or *dictionary-encoded* representation. This representation can yield significant performance improvements when doing analytics.

## The Categorical Type

In [9]:
fruits = 'apple orange apple apple'.split()*2
fruits

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']

In [10]:
N = len(fruits)

In [11]:
df = pd.DataFrame({
    'fruit': fruits,
    'basket_id': np.arange(N),
    'count': np.random.randint(3,15, size=N),
    'weight': np.random.uniform(0, 4, size=N)},
    columns=['basket_id', 'fruit', 'count', 'weight'])

df

,basket_id,fruit,count,weight
0,0,apple,8,2.592884
1,1,orange,9,1.346389
2,2,apple,8,0.004594
3,3,apple,3,3.314898
4,4,apple,5,2.576201
5,5,orange,5,2.420668
6,6,apple,12,1.982603
7,7,apple,12,3.835266


In [12]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [13]:
c = fruit_cat.values
type(c)

pandas.core.categorical.Categorical

In [14]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [15]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

So we have a category. You can extract the values from the category and you have a categorical. The categorical can give you the categories and the codes.

In [16]:
# Assignment

df['fruit'] = df['fruit'].astype('category')

In [17]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [18]:
df['fruit'].values.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

`pd.Categorical` from other Python types is possible.

In [19]:
my_categories = pd.Categorical('foo bar baz foo bar'.split())
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

### Reverse: codes to categories

In [20]:
categories = 'foo bar baz'.split()
codes = [0, 1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [21]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

### Ordered categories

In [22]:
ordered_cat = pd.Categorical.from_codes(
    codes,
    categories,
    ordered=True)

ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

## Computation with `Categorical`

### Also binning

In [23]:
np.random.seed(12345)

In [24]:
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [25]:
bins = pd.qcut(draws, 4, labels='Q1 Q2 Q3 Q4'.split())

In [26]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [27]:
bins = pd.Series(bins, name='quartile')

In [28]:
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [29]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### Performance boost

In [30]:
N = 1_000_000

draws = pd.Series(np.random.randn(N))

In [31]:
labels = pd.Series('foo bar baz qux'.split() * (N // 4))

In [32]:
categories = labels.astype('category')

In [33]:
def printc(num):
    """Print number with commas"""
    print(f'{num:,}')

In [34]:
printc(labels.memory_usage())

8,000,080


In [35]:
printc(categories.memory_usage())

1,000,272


In [36]:
%time _ = labels.astype('category')

CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 36.9 ms


The conversion to `Categorical` is a one time cost. Thereafter many operations are much faster. `groupby` operations will use the integer-based `codes` array instead of an array of strings.

## Categorical Methods

In [37]:
s = pd.Series('a b c d'.split() * 2)
cat_s = s.astype('category')

In [38]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

There is the special attribute `cat` in categorical `Series` objects.

In [39]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

And you can do manual modifications to the categories.

In [40]:
actual_categories = 'a b c d e'.split()

In [41]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [42]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [43]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [44]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

See the fifth category there? Nice.

Now check out this filter... and subsequently the removal of unused categories.

In [45]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [46]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

### Dummy variables, aka one-hot encoding

In [47]:
cat_s = pd.Series('a b c d'.split() * 2, dtype='category')

In [48]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


Are you kidding me?? It's that easy?